### 作業目的: 使用樹型模型進行文章分類

本次作業主利用[Amazon Review data中的All Beauty](https://nijianmo.github.io/amazon/index.html)來進行review評價分類(文章分類)

資料中將review分為1,2,3,4,5分，而在這份作業，我們將評論改分為差評價、普通評價、優良評價(1,2-->1差評、3-->2普通評價、4,5-->3優良評價)

### 載入套件

In [1]:
import json
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

### 資料前處理
文本資料較為龐大，這裡我們取前10000筆資料來進行作業練習

In [5]:
#load json data
all_reviews = []
with open('All_Beauty.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        all_reviews.append(json.loads(line))

all_reviews[0]

{'overall': 1.0,
 'verified': True,
 'reviewTime': '02 19, 2015',
 'reviewerID': 'A1V6B6TNIC10QE',
 'asin': '0143026860',
 'reviewerName': 'theodore j bigham',
 'reviewText': 'great',
 'summary': 'One Star',
 'unixReviewTime': 1424304000}

In [8]:
#parse label(overall) and corpus(reviewText)
corpus = []
labels = []

for review in all_reviews[:10000]:
    #print(review)
    #transform labels: 1,2 --> 1 and 3 --> 2 and 4,5 --> 3
    try:
        corpus.append(review['reviewText'])
        if review['overall'] <= 2:
            labels.append(1)
        elif review['overall'] == 3:
            labels.append(2)
        else:
            labels.append(3)
    except:
        pass


In [9]:
from nltk.corpus import stopwords

import nltk

nltk.download('stopwords')

# Lemmatize with POS Tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

## 創建Lemmatizer
lemmatizer = WordNetLemmatizer() 
def get_wordnet_pos(word):
    """將pos_tag結果mapping到lemmatizer中pos的格式"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def clean_content(X):
    stop_words = set(stopwords.words('english'))
    content_clean = []
    # remove non-alphabet characters
    X_clean = re.sub('[^a-zA-Z]',' ', X).lower()
    # tokenize
    X_word_tokenize = nltk.word_tokenize(X_clean)

    # stopwords_lemmatizer
    for word in X_word_tokenize:
        if word not in stop_words:
            word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
            content_clean.append(word)
                
    return ' '.join(content_clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#preprocessing data
#remove email address, punctuations, and change line symbol(\n)

corpus = [clean_content(review) for review in corpus]

In [18]:
#split corpus and label into train and test
x_train, x_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.2)

len(x_train), len(x_test), len(y_train), len(y_test)

(7996, 1999, 7996, 1999)

In [19]:
#change corpus into vector
#you can use tfidf or BoW here
tfidf = TfidfVectorizer(max_features=3600)
tfidf.fit(x_train)

print(tfidf.transform(x_test).toarray())
#transform training and testing corpus into vector form
x_train = tfidf.transform(x_train).toarray()
x_test = tfidf.transform(x_test).toarray()

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### 訓練與預測

In [37]:
#build classification model (decision tree, random forest, or adaboost)
#start training
cls = RandomForestClassifier(n_estimators=66,
                             class_weight={1: 20, 2: 10, 3:1})
cls.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight={1: 20, 2: 10, 3: 1},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=66, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [38]:
#start inference
y_pred = cls.predict(x_test)

In [39]:
#calculate accuracy
a = sum(y_test==y_pred) / len(y_pred)
print('Accuracy:', a)

Accuracy: 0.8999499749874937


In [40]:
#calculate confusion matrix, precision, recall, and f1-score
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.66      0.17      0.27       138
           2       0.44      0.05      0.10        75
           3       0.91      0.99      0.95      1786

   micro avg       0.90      0.90      0.90      1999
   macro avg       0.67      0.40      0.44      1999
weighted avg       0.87      0.90      0.87      1999

[[  23    1  114]
 [   2    4   69]
 [  10    4 1772]]


由上述資訊可以發現, 模型在好評的準確度高(precision, recall都高), 而在差評的部分表現較不理想, 在普通評價的部分大部分跟差評搞混,
同學可以試著學習到的各種方法來提升模型的表現